In [41]:
from numpy import *
from pylab import *
from math import erf
import pandas as pd

In [42]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [43]:
# constant cell

r = 0.07
sigma = 0.09
k = 120

N = 70
s_start = 0
s_end = 490
s = linspace(s_start, s_end, N + 1)
ds = abs(s_end - s_start)/N  # ds, uniform grid

M = 100
t_start = 0
t_end = 1  # T
time = 0
dt = abs(t_end - t_start)/M  # dt, uniform grid

In [44]:
# function cell for Euler's method

V = [0 for i in range(N + 1)]  # exact solution V(s, T-t)
W = [0 for i in range(N + 1)]  # numerical solution W(s, T-t)
b = [0 for i in range(N + 1)]
c = [0 for i in range(N + 1)]  # coefficient list
row_vector = [0 for i in range(N + 1)]
v_vector = [0 for i in range(N + 1)]
row_vector[0] = ds; row_vector[1] = 4*ds; row_vector[2] = ds;  # consider N > 2
spl_matrix = []
s_second_partial = [0 for i in range(N + 1)]
s_first_partial = [0 for i in range(N + 1)]

In [45]:
def s_(i):
    return s_start + i*ds

def spline(s, i):
    return W[i] + ((W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1]))*(s-s_(i)) + c[i]*(s-s_(i))**2 + (c[i+1]-c[i])/(3*ds)*(s-s_(i))**3

def rotate(arr, n):
    n %= len(arr)
    if not n:
        return arr
    
    left = arr[:-n]
    right = arr[-n:]
    
    return right + left

def normal_dist(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def CDF(t, idx):
    arr = [0 for i in range(N + 1)]
    for i in range(1, N+1):
        if idx == 1:
            arr[i] = normal_dist((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
        else:
            arr[i] = normal_dist((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            
    return arr

def exact_func(t):
    return s*float_(CDF(t,1)) - k*exp(-r*t)*float_(CDF(t,2))

In [46]:
row_end = [0 for i in range(N + 1)]
row_end[0] = 1
spl_matrix.append(row_end)
spl_matrix.append(row_vector)

for i in range(2,N):
    row_vector = rotate(row_vector,1)
    spl_matrix.append(row_vector)
    
row_end = [0 for i in range(N + 1)]
row_end[N] = 1
spl_matrix.append(row_end)

1 0 0 0 
163.33333333333334 653.3333333333334 163.33333333333334 0 
0 163.33333333333334 653.3333333333334 163.33333333333334 
0 0 0 1 
400166.666666667


In [99]:
time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
title('W(x) at time = ' + str(round(time, 2)))
legend()
axis((90, 150, 0, 45))
pause(1)

while time < t_end:
    time = time + dt
    
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    W = W + dt*(r*float_(s)*float_(b) + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)

    clf()
    plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
    plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
    title('W(x) at time = ' + str(round(time, 2)))
    axis((90, 150, 0, 45))
    legend()
    pause(0.001)